#### Copyright 2020 Google LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video Classification with Pre-Trained Models Project

In this project we will import a pre-existing model that recognizes objects and use the model to identify those objects in a video. We'll edit the video to draw boxes around the identified object, and then we'll reassemble the video so the boxes are shown around objects in the video.

# Exercises

## Exercise 1: Coding

You will process a video frame by frame, identify objects in each frame, and draw a bounding box with a label around each car in the video.
 
Use the [SSD MobileNet V1 Coco](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md) (*ssd_mobilenet_v1_coco*) model. The video you'll process can be found [on Pixabay](https://pixabay.com/videos/cars-motorway-speed-motion-traffic-1900/). The 640x360 version of the video is smallest and easiest to handle, though any size should work since you must scale down the images for processing.
 
Your program should:
 
* Read in a video file (use the one in this colab if you want)
* Load the TensorFlow model linked above
* Loop over each frame of the video
* Scale the frame down to a size the model expects
* Feed the frame to the model
* Loop over detections made by the model
* If the detection score is above some threshold, draw a bounding box onto the frame and put a label in or near the box
* Write the frame back to a new video
 
Some tips:
 
* Processing an entire video is slow, so consider truncating the video or skipping over frames during development. Skipping frames will make the video choppy. But you'll be able to see a wider variety of images than you would with a truncated video with all of the original frames in the clip.
* The model expects a 300x300 image. You'll likely have to scale your frames to fit the model. When you get a bounding box, that box is relative to the scaled image. You'll need to scale the bounding box out to the original image size.
* Don't start by trying to process the video. Instead, capture one frame and work with it until you are happy with your object detection, bounding boxes, and labels. Once you get those done, use the same logic on the other frames of the video.
* The [Coco labels file](https://github.com/nightrome/cocostuff/blob/master/labels.txt) can be used to identify classified objects.
 

### **Student Solution**

In [ ]:
# Your code goes here
import cv2 as cv
from PIL import Image as im
import tensorflow as tf
import urllib.request
import os
import tarfile
import shutil
import matplotlib.pyplot as plt

cars_video = cv.VideoCapture('cars.mp4')

height = int(cars_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(cars_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = cars_video.get(cv.CAP_PROP_FPS)
total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

print(f'height: {height}')
print(f'width: {width}')
print(f'frames per second: {fps}')
print(f'total frames: {total_frames}')
print(f'video length (seconds): {total_frames / fps}')

cars_video.release()

cars_video = cv.VideoCapture('cars.mp4')
total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

frames_read = 0

for current_frame in range(0, total_frames):
  cars_video.set(cv.CAP_PROP_POS_FRAMES, current_frame)
  ret, _ = cars_video.read()
  if not ret:
    raise Exception(f'Problem reading frame {current_frame} from video')
  if (current_frame+1) % 50 == 0:
    print(f'Read {current_frame+1} frames so far')

cars_video.release()

print(f'Read {total_frames} frames')

base_url = 'http://download.tensorflow.org/models/object_detection/'
file_name = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'

url = base_url + file_name

urllib.request.urlretrieve(url, file_name)

#os.listdir()

dir_name = file_name[0:-len('.tar.gz')]

if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')

# os.listdir(dir_name)

frozen_graph = os.path.join(dir_name, 'frozen_inference_graph.pb')

with tf.io.gfile.GFile(frozen_graph, "rb") as f:
  graph_def = tf.compat.v1.GraphDef()
  loaded = graph_def.ParseFromString(f.read())


outputs = (
  'num_detections:0',
  'detection_classes:0',
  'detection_scores:0',
  'detection_boxes:0',
)

def wrap_graph(graph_def, inputs, outputs, print_graph=False):
  wrapped = tf.compat.v1.wrap_function(
    lambda: tf.compat.v1.import_graph_def(graph_def, name=""), [])
  return wrapped.prune(
    tf.nest.map_structure(wrapped.graph.as_graph_element, inputs),
    tf.nest.map_structure(wrapped.graph.as_graph_element, outputs)
  )
    
model = wrap_graph(graph_def=graph_def, inputs=["image_tensor:0"], outputs=outputs)

#input video
input_video = cv.VideoCapture('cars.mp4')

class_dict = {
    3.0: 'car',
    6.0: 'bus',
    7.0: 'train',
    8.0: 'truck',
    77.0: 'cell phone'
}

#getting height width fps
height = int(input_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(input_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = input_video.get(cv.CAP_PROP_FPS)

#generate video file to be written
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('processed.mp4', fourcc, fps, (300, 300))


for i in range(0, int(input_video.get(cv.CAP_PROP_FRAME_COUNT)), 5):
  input_video.set(cv.CAP_PROP_POS_FRAMES, i)
  ret, frame = input_video.read()
  if ret == True:
    processed = cv.resize(frame,(width, height),fx=0,fy=0, interpolation = cv.INTER_CUBIC)
    output_video.write(processed)
    output_video.write(frame)

  tensor = tf.convert_to_tensor([processed], dtype=tf.uint8)
  detections = model(tensor)

  num_pet = int(detections[0].numpy()[0])
  classes = detections[1].numpy()[0, :num_pet]
  scores = detections[2].numpy()[0, :num_pet]
  boxes = detections[3].numpy()[0, :num_pet]
  color = None
  H, W, _ = processed.shape
  for x in range(num_pet):
    if scores[x] > 0.5:
      box = boxes[x]
      y1, x1, y2, x2 = box
      x1 *= W 
      x2 *= W 
      y1 *= H
      y2 *= H
      cv.rectangle(processed, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
      cv.putText(processed, class_dict[classes[x]], (int(x1), int(y2+30)), cv.FONT_HERSHEY_SIMPLEX, 1, [255, 0, 0], 2) 
    output = cv.resize(processed, (300, 300))

  output_video.write(output)
input_video.release()
output_video.release()

---

## Exercise 2: Ethical Implications

Even the most basic models have the potential to affect segments of the population in different ways. It is important to consider how your model might positively and negatively affect different types of users.

In this section of the project, you will reflect on the positive and negative implications of your model. Frame the context of your model creation using this narrative:

> The city of Seattle is attempting to reduce traffic congestion in its downtown area. As part of this project, they plan to allow each local driver one free trip to downtown Seattle per week. After that, the driver will have to pay a $50 toll for each extra day per week driven. As an early proof of concept for this project, your team is tasked with using machine learning to correctly identify automobiles on the road. The next phase of the project will involve detecting license plate numbers and then cross-referencing that data with RFID chips that should be mounted in all local drivers' cars.

### **Student Solution**

**Positive Impact**

Your model is trying to solve a problem. Think about who will benefit from that problem being solved and write a brief narrative about how the model will help.

> *Hypothetical entities will benefit because...*
the model could benefitthe traffic association to count the traffic flow improve the traffic flow at the heavy-traffic locations, throughout different times of the day. As well as the police in terms of speedint regulation and tickets.


**Negative Impact**

Models rarely benefit everyone equally. Think about who might be negatively impacted by the predictions your model is making. This person(s) might not be directly using the model, but they might be impacted indirectly.

> *Hypothetical entities will be negatively impacted because...*
This model will have a negative  impact on the drivers who consitenly drive over the speed limit. Or even one-off situations where someone temporarily goes over the speed limit.



**Bias**

Models can be biased for many reasons. The bias can come from the data used to build the model (e.g., sampling, data collection methods, available sources) and/or from the interpretation of the predictions generated by the model.

Think of at least two ways bias might have been introduced to your model and explain both below.

> *One source of bias in the model could be...One bias that we notices is that not every vehicle is detected in our output, which could lead to a lower accuracy.*

> *Another source of bias in the model could be...A second bias that we noticied is incorrect classifying, as our model misidentified some of our cars.*

**Changing the Dataset to Mitigate Bias**

Having bias in your dataset is one of the primary ways in which bias is introduced to a machine learning model. Look back at the input data you fed to your model. Think about how you might change something about the data to reduce bias in your model.

What change or changes could you make to reduce the bias in your dataset? Consider the data you have, how and where it was collected, and what other sources of data might be used to reduce bias.

Write a summary of changes that could be made to your input data.

> *Since the data has potential bias A we can adjust... I think we could just add more training data or have a smaller, less traffic video to allow our program to be more accurate and precise.*

**Changing the Model to Mitigate Bias**

Is there any way to reduce bias by changing the model itself? This could include modifying algorithmic choices, tweaking hyperparameters, etc.

Write a brief summary of changes you could make to help reduce bias in your model.

> *Since the model has potential bias A, we can feed in training videos that contain individual colors so that the trained model can better identify and classify(box every moving objects correctly) cars.*

**Mitigating Bias Downstream**

Models make predictions. Downstream processes make decisions. What processes and/or rules should be in place for people and systems interpreting and acting on the results of your model to reduce bias? Describe these rules and/or processes below.

> *Since the predictions have potential bias A, we can adjust...*
Since the predictions have potential bias in favored to specific cars with specific colors, then we can come up a way to preprocess the videoes so that different color cars can be correctly exposed in the videoes for training. one potential way to preprocess the videoes is to adjust the threshold to expose different colors.

---